In [ ]:
import xlrd
import os.path
import random
import numpy as np
import pandas as pd
import math
import random
from collections import defaultdict
import time
from random import shuffle

# File Reading

In [ ]:
df = pd.read_csv(os.path.join('/Users/firzoknadeem/Desktop/University Files/AI/Project','Test Case 1.csv') , header=None)

In [ ]:
df_obj = df.select_dtypes(['object'])

df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [ ]:
df

# Data Extraction

In [ ]:
totalColumns = df.shape[1]

In [ ]:
#searching for the starting of the row data
rowStart = 0
courseStart='C1'
found = False
for i in range(0, df.shape[0]):
    for j in range(df.shape[1]):
        val = df.values[i][j]
        if val == courseStart:
            found = True
            break
    if found:
        break
rowStart = i
print ("rowStart", i) #---> sample data starting from this row

#getting the courses names
courses = []
noCourses = totalColumns-1
for k in range(totalColumns):
    val = df.values[rowStart, k]
    if val == val:#Check for nan
        courses.append(val)
print ("courses", courses)
rowStart += 1
print ("rowStart", rowStart) #---> sample data starting from this row

In [ ]:
#finding exact number of rows that have the value i.e. data of corresponding teacher and section
totalRows = 0 #----> has the total number of rows that contains data
rowList = [] #contains the integer values of rows
p = 'Preferences'
found = False
for i in range(rowStart, df.shape[0]):
    for j in range(df.shape[1]):
        val = df.values[i][j]
        if val == p:
            found = True
    if found:
        break
    rowList.append(i)

totalRows = len(rowList)
print ("totalRows", totalRows)
print ("rowList", rowList)

In [ ]:
totalClassesInWeek = int(df.values[0, 1])

teacherCount = int(df.values[1, 1])

roomCount = int(df.values[2, 1])

groupCount = int(df.values[3, 1])

In [ ]:
#Getting all teachers preferences in tuples

startPreferenceIndex = df.loc[df[0] == 'Preferences'].index[0] + 1    # getting the starting index of teacher's preference
teacherPreference = {}
for i in range(startPreferenceIndex,df.shape[0]):
    teacherPreference[df.values[i,0]] = df.values[i,1]

# Creation of Chromosome

In [ ]:
class gene:
    def __init__(self, day, room, time):
        self.day = day
        self.room = room
        self.time = float(time)

In [ ]:
class slot:# Used as a hash table e.g BatchCS16_A$C1$T1$1
    def __init__(self, group, course, teacher, duration):
        self.group = group
        self.course = course
        self.teacher = teacher
        self.duration = float(duration)

In [ ]:
#hashtable for chromosomes
#all possible classes for groups and courses
slots = []
for i in range(rowStart, rowStart + totalRows, 2):

    for j, c in enumerate(courses):
#         Group Course Teacher Credit Hour
        val = slot(str(df.values[i,0].strip()), str(c), str(df.values[i, j+1].strip()), str(df.values[i+1, j+1]))
        slots.append(val)
        slots.append(val)
        
        if float(df.values[i+1, j+1])==1:
            slots.append(val)

In [ ]:
#Possible values for chromosomes e.g Mon_1_8.5

days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
rooms = list(range(1, roomCount+1))
startTime = np.arange(9, 17, 1)

day_room_time = []
for d in days:
    for r in rooms:
        for s in startTime:
            
            if ((d == 'Fri') and (s == 13)):#Dont add the timings of Friday Prayer (CSP)
                continue
            
            val = gene(day=d, room=r, time=s)

            day_room_time.append(val)

# Initial population

In [ ]:
#First population
#Now randomly placing classes in chromosome and adding to population array
def generateFirstPopulation(populationSize = 500):
    population = []
    possibleValues = day_room_time.copy()
    for i in range(populationSize):
        chromosome = []
        possibleValues = day_room_time.copy()
        for j in range(len(slots)):
            chromosome.append(possibleValues.pop(random.randint(0, len(possibleValues)-1)))
        population.append(chromosome)
    return population

In [ ]:
# Getting all indexes in hashtable for all teachers

teacherSlots = defaultdict(list)
for i in range(0,len(slots)):

    teacherSlots[slots[i].teacher].append(i)

In [ ]:
# Used to display number and type of constraints 
hardConstraintsDict = defaultdict(int)
softConstraintsDict = defaultdict(int)

In [ ]:
dayConstraint = "Fri"
dayConstraintGroup = 'BatchCS16_A'

# Fitness Function

In [ ]:

def calculateFitness(chromosome, checkSoftConstraint = True, checkTeacherPreference = True):
    
    softConstraintDelta = 1
    hardConstraintDelta = 5
    
    fitness = 5000
    
    for i in range(len(chromosome)):
        
        chromo = chromosome[i]
    
        day = chromo.day
        room = chromo.room
        startTime = chromo.time

        #Teacher constraints
        
        s = slots[i]

        teacher = s.teacher
        duration = s.duration
        group = s.group
        teaches = teacherSlots[teacher] #getting list of slots in which teacher teaches
        
        teacherPreferenceVoilation = 0
        teacherClashesVoilation = 0
        teacherConsecutiveVoilation=0
        RoomClashesVoilation = 0
        FridayVoilation = 0
        
        try:
            if teacherPreference[teacher] == 'm':
                if startTime + duration >= 13:
                    
                    softConstraintsDict["teacherPreferenceVoilation"] +=1
                    fitness -= softConstraintDelta
                    
            elif teacherPreference[teacher] == 'e':
                if startTime + duration <= 13:

                    softConstraintsDict["teacherPreferenceVoilation"] +=1
                    fitness -= softConstraintDelta
                    
        except:
            pass
        
        for t in teaches:
            if t != i:#so that it does not test with itself
                
                #test data
                testChromo = chromosome[t]
                
                tday = testChromo.day
                troom = testChromo.room
                tstartTime = testChromo.time
            
                if tday == day:
                    if tstartTime == startTime: #teacher cant be at two places at the same time
                        
                        fitness -= hardConstraintDelta
                        hardConstraintsDict["teacherClashesVoilation"] += 1
                        
                    elif (startTime + duration == tstartTime) and checkSoftConstraint: #consecutive classes for teacher
                        
                        fitness -= softConstraintDelta
                        softConstraintsDict["teacherConsecutiveVoilation"] += 1
        
        
        #Room constraints
        
        for j in range(len(chromosome)):
            if i != j:
                #test data
                testChromo = chromosome[j]
                testSlots= slots[j]
                
                tday = testChromo.day
                troom = testChromo.room
                tstartTime = testChromo.time
                
                tgroup = testSlots.group
                tteacher = testSlots.teacher
                tduration = testSlots.duration
                
                if tday == day:
                    if (tgroup == group):
                        if (tstartTime == startTime):
                            fitness -= hardConstraintDelta
                            hardConstraintsDict["SameSectionClashes"] +=1
                    if troom == room:
                        if (startTime <= tstartTime < (startTime + duration)):
                            fitness -= hardConstraintDelta
                            hardConstraintsDict["TimeClashesVoilation"] +=1
                            
        
        #Friday Prayer Time 13-14
        if day == "Fri":
            if 13 <= startTime < 14:
                fitness -= hardConstraintDelta
                hardConstraintsDict["FridayVoilation"] +=1
#                 FridayVoilation = FridayVoilation + 1
        
#         if day == dayConstraint and group == dayConstraintGroup:
#             softConstraintsDict["DayConstraint"] += 1
#             fitness -= hardConstraintDelta
        
        
            
    return fitness/5000   #return a probabilistic value

# Crossover Function

In [ ]:
def crossover(chromosome1, chromosome2):
    
    child1 = []
    child2 = []
    for i in range(len(chromosome1)):
        
        if random.randint(0,1) == 1:
            child1.append(chromosome2[i])
            child2.append(chromosome1[i])
        else:
            child1.append(chromosome1[i])
            child2.append(chromosome2[i])
    
    
    return child1, child2

In [ ]:
# 2 Point Crossover function
# def crossover(chromosome1, chromosome2):
    
#     splits = random.sample(range(1, len(chromosome1)-1), 2)
#     splits=sorted(splits)
#     child1 = chromosome1[0:splits[0]]
#     child2 = chromosome2[0:splits[0]]
    
#     child1 = np.hstack([child1, chromosome2[splits[0]:splits[1]]])
#     child2 = np.hstack([child2, chromosome1[splits[0]:splits[1]]])

    
#     child1 = np.hstack([child1, chromosome1[splits[1]:]])
#     child2 = np.hstack([child2, chromosome2[splits[1]:]])  
    
#     return child1, child2

# Mutation function

In [ ]:
def mutation(chromosome, levelOfMutation = 0.01):
    
    returnChromo = chromosome.copy()
    for i in range(len(returnChromo)):
        
        if random.uniform(0, 1) < levelOfMutation:
            returnChromo[i] = day_room_time[random.randint(0, len(day_room_time)-1)]
            
    return returnChromo

In [ ]:
def nextGeneration(chromosome1, chromosome2):
    child1, child2 = crossover(chromosome1, chromosome2)
    
    if random.uniform(0, 1) < 0.3:
        child1 = mutation(child1)
    if random.uniform(0, 1) < 0.3:
        child2 = mutation(child2)
    
    
    return child1, child2
    

# Genetic Algorithm

In [ ]:
t1 = time.time()
population = generateFirstPopulation(populationSize=100)
oldmaxFitness = -5000000
newmaxFitness = -5000000

generation = 0
population.extend(bestPopulation)
while True:
    t2 = time.time()
    generation+=1
    print(generation)

    fitnesses = []
    for i in range(len(population)):
        fitnesses.append(calculateFitness(population[i]))
    fitnessinds = np.argsort(fitnesses)
    sortedFitness = np.array(fitnesses)[fitnessinds][::-1]
    sortedPopulation = np.array(population)[fitnessinds][::-1]

    bestPopulation = sortedPopulation[0:50]

    oldmaxFitness = newmaxFitness

    newmaxFitness = sortedFitness[0]

    print(sortedFitness[:10])
    if (newmaxFitness >= 0.999):
        print(newmaxFitness)
        break
    
    nextPopulation = []
    nextPopulation.extend(bestPopulation[0:10])
    shuffle(bestPopulation)
    for i in range(0, len(bestPopulation), 2):
        a, b = nextGeneration(bestPopulation[i], bestPopulation[i+1])
        nextPopulation.append(a)
        nextPopulation.append(b)
        nextPopulation.append(mutation(bestPopulation[i] , levelOfMutation = 0.2))
    
    
    #sort the randoms and then add
    f = []
    newpop = generateFirstPopulation(populationSize=100)
    for i in range(len(newpop)):
        f.append(calculateFitness(newpop[i]))
    fitnessinds = np.argsort(f)
    sortedFitness = np.array(f)[fitnessinds][::-1]
    sortedPopulation = np.array(newpop)[fitnessinds][::-1]
    nextPopulation.extend(sortedPopulation[:10])

    #add old best population by only doing mutation
    population = nextPopulation
    print("generation "+str(generation)+" took time: "+str(time.time()-t2))


print("Total time: "+str(time.time()-t1))
# Total time: 5329.4181208610535 old 669 generation

In [ ]:
c = bestPopulation[0]
print(calculateFitness(bestPopulation[0]))


In [ ]:
day_room_time[0].time

In [ ]:
for item_a, i in zip(slots, c):
    print(item_a.group, item_a.course, item_a.teacher, item_a.duration)
    print(i.day,i.room,i.time)

# Generate readable Timetable

In [ ]:
def makeTimetable(chromosomes):
    
    roomNumbers = []
    for i in range(roomCount):
        roomNumbers.append(str(i+1))
    
    print(roomNumbers)
    timetableArray =np.array([["______________________"]*(len(startTime))]*roomCount*len(days))
    print(timetableArray.shape)


    for classIdx in range(0,len(chromosomes)):
        
        day = days.index(chromosomes[classIdx].day)

        room = roomNumbers.index(str(chromosomes[classIdx].room))

        time = np.where((startTime) == float(chromosomes[classIdx].time))[0][0]
       
        if timetableArray[(day*15)+room][time] == '______________________':
            group = slots[classIdx].group
            course = slots[classIdx].course
            duration = slots[classIdx].duration
            timetableArray[(day*15)+room][time] = str(group) + "_" + str(course)+ "_" + str(duration)

    return timetableArray
Timetable = makeTimetable(bestPopulation[0])
print(calculateFitness(bestPopulation[0]))



In [ ]:
df=pd.DataFrame(data = Timetable)
indice = []
for i in range(roomCount):
    indice.append("Room "+ str(i+1))
print(indice)
indices = indice * 5 
df.index = indices

df.columns = ["9:00-10:00","10:00-11:00","11:00-12:00","12:00-1:00","1:00-2:00","2:00-3:00","3:00-4:00","4:00-5:00"]


In [ ]:
df.to_csv("TimeTable 5.csv")

In [ ]:
soft_Constraints=pd.DataFrame(data = softConstraintsDict, index=["Soft Constraints"])
soft_Constraints.to_csv("Soft Constraints.csv")

hard_Constraints=pd.DataFrame(data = hardConstraintsDict, index=["Hard Constraints"])
hard_Constraints.to_csv("Hard Constraints.csv")